In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.datasets import imdb


Using Theano backend.


In [2]:
import string
from nltk.stem.porter import *
import multiprocessing
import unicodedata
import nltk
from joblib import Parallel, delayed
import tqdm
import multiprocessing
import os
import glob

In [3]:
def load_aclImdb_data(path_train, path_test):
    sentences_train = []
    train_y = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path_train)
    for ff in glob.glob("*.txt"):
       	with open(ff, 'r') as f:
	    sentences_train.append(f.readline().strip())
            train_y.append(1)
    os.chdir('%s/neg/' % path_train)
    for ff in glob.glob("*.txt"):
       	with open(ff, 'r') as f:
            sentences_train.append(f.readline().strip())
            train_y.append(0)
    os.chdir(currdir)
    sentences_test = []
    test_y = []
    currdir = os.getcwd()
    os.chdir('%s/pos/' % path_test)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences_test.append(f.readline().strip())
            test_y.append(1)
    os.chdir('%s/neg/' % path_test)
    for ff in glob.glob("*.txt"):
        with open(ff, 'r') as f:
            sentences_test.append(f.readline().strip())
            test_y.append(0)
    os.chdir(currdir)
    return sentences_train, sentences_test, train_y, test_y
X_train, X_test, y_train, y_test = load_aclImdb_data("/Users/zacharie/Documents/AIC/REI-Option/aclImdb/train", "/Users/zacharie/Documents/AIC/REI-Option/aclImdb/test")

In [4]:
def stemmering_sentences(sentence):
    # Remove punctuation, stopword and then stemmering
    punctuation = set(string.punctuation)
    stemmer = nltk.PorterStemmer()
    tmp = unicode(sentence, errors='ignore')
    doc = [stemmer.stem(word.lower()) for word in nltk.word_tokenize(tmp) if
           (word not in punctuation) and (word not in nltk.corpus.stopwords.words('english')) and (word != 'br')]
    return doc

def stemmering_sentences_imdb(X, y):
    sentences_stem = Parallel(n_jobs=4)(delayed(stemmering_sentences)(sentence) for sentence in tqdm.tqdm(X, desc="stem"))
    return sentences_stem, y
X, y = stemmering_sentences_imdb(X_train+X_test, y_train+y_test)


stem: 100%|██████████| 50000/50000 [10:52<00:00, 76.59it/s]

stem: 100%|█████████▉| 49998/50000 [10:52<00:00, 62.38it/s]

stem: 100%|█████████▉| 49986/50000 [10:52<00:00, 62.67it/s]

stem: 100%|█████████▉| 49978/50000 [10:52<00:00, 67.85it/s]

stem: 100%|█████████▉| 49968/50000 [10:52<00:00, 86.33it/s]

...

stem:   0%|          | 66/50000 [00:00<10:55, 76.15it/s]

stem:   0%|          | 48/50000 [00:00<13:02, 63.86it/s]

stem:   0%|          | 22/50000 [00:00<16:54, 49.26it/s]

stem:   0%|          | 8/50000 [00:00<21:16, 39.16it/s]

stem:   0%|          | 0/50000 [00:00<?, ?it/s]

In [5]:
# combine_train_test_X = terms_by_doc_train + terms_by_doc_test
combine_train_test_X = X


In [6]:
vocab_dim = 50
maxlen = 400
n_iterations = 1  # ideally more..
n_exposures = 30
window_size = 5
batch_size = 200
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2
cpu_count = multiprocessing.cpu_count()

In [7]:
from gensim.models.word2vec import Word2Vec
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 window=window_size,
                 workers=cpu_count,
                 iter=n_iterations)
model.build_vocab(combine_train_test_X)
model.train(combine_train_test_X)


5806352

Training a Word2vec model...


In [8]:
from gensim.corpora.dictionary import Dictionary
gensim_dict = Dictionary()
gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# gensim_dict.items() returns [(0, u"'surpris"), (1, u'woodi'), (2, u'yellow'),...]
# K+1 aims at avoiding 0 as index.
w2indx = {v: k+1 for k, v in gensim_dict.items()}
w2vec = {word: model[word] for word in w2indx.keys()}
# print len(model["surpris"]) -> 100
print('Setting up Arrays for Keras Embedding Layer...')
n_symbols = len(w2indx) + 1  # adding 1 to account for 0th index
embedding_weights = np.zeros((n_symbols + 1, vocab_dim))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]


Setting up Arrays for Keras Embedding Layer...


In [9]:
from sklearn.model_selection import train_test_split
terms_by_doc_train, terms_by_doc_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=58)
X_train = []
for doc in terms_by_doc_train:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_train.append(new_txt)
X_test = []
for doc in terms_by_doc_test:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_test.append(new_txt)

In [11]:

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(input_dim = n_symbols + 1,
                    output_dim = vocab_dim,
                    input_length=maxlen,
                    dropout=0.2))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))
print("Evaluate...")
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


Test score: 0.276048250993
Test accuracy: 0.888399996758


15000/15000 [==============================] - 15s    

14800/15000 [============================>.] - ETA: 0s

14600/15000 [============================>.] - ETA: 0s

14400/15000 [===========================>..] - ETA: 0s

...

  800/15000 [>.............................] - ETA: 14s

  600/15000 [>.............................] - ETA: 14s

  400/15000 [..............................] - ETA: 15s

  200/15000 [..............................] - ETA: 15s

Evaluate...


In [12]:
from keras.models import load_model
import pickle
model.save('./cnn_train_imdb.h5')
import pickle
with open("./cnn_w2indx_dict_imdb.pkl", 'wb') as f:
    pickle.dump(w2indx, f)

In [16]:
with open("./cnn_w2indx_dict_imdb.pkl", 'rb') as f:
    w2indx_load = pickle.load(f)
model = load_model('cnn_train_imdb.h5')

In [17]:
demo = ["It is one good film.", "He said it is one good film, but he is wrong."]
demo = [stemmering_sentences(i) for i in demo]
X_demo = []
for doc in demo:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx_load[word])
        except:
            new_txt.append(0)
    X_demo.append(new_txt)

In [21]:
X_demo

[[6401, 989, 7654, 7391], [5807, 6317, 989, 7654, 7391, 973]]

In [25]:
X_demo = sequence.pad_sequences(X_demo, maxlen=maxlen)
model.predict(X_demo)

array([[ 0.91695625],
       [ 0.80726594]], dtype=float32)